In [11]:
import numpy as np
import tensorflow as tf
import os

load training data from .dat files
raw_data_imaginary_x.dat --> train_imag
raw_data_real_x.dat      --> train_real

In [51]:
# if IS_TRAINING == True, a new model will be compiled 
# and trained from the dataset
#
# if IS_TRAINING == False, an existing model will be 
# loaded and tested against the dataset
IS_TRAINING = True

# trainingset to be trained or loaded
trainingset_number = 5

# training params
neuron_number = 8
activation_function = 'tanh'
iterations = 20000
learning_rate= 1e-4

checkpoint_path = "./training_" + str(trainingset_number) + "/cp_training_weights_only.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

os.mkdir("./training_" + str(trainingset_number))



In [13]:
# filename_raw_data_real = "raw_data_real_" + str(trainingset_number) + ".dat"
# filename_raw_data_imag = "raw_data_imaginary_" + str(trainingset_number) + ".dat"

filename_dataset_csv = "training_data_" + str(trainingset_number) + ".csv"


dataset_array_numpy = np.loadtxt(filename_dataset_csv,
  dtype=float, 
  delimiter=',',
  skiprows=1) # ignore header line




In [7]:
dataset = tf.data.experimental.CsvDataset(
  filename_dataset_csv,
  [tf.float32,  # Required field, use dtype or empty tensor
   tf.float32,
   tf.float32,
   tf.float32
  ],
  header=True,
  select_cols=[0,1,2,3]  
)

In [ ]:
def show_dataset_structure():

  print(dataset)
  i = 0

  for element in dataset.as_numpy_iterator():
    print(element)
    i = i + 1
    if i > 10:
        break

show_dataset_structure()

In [ ]:
# transform dataset
def transform_func(element1, element2, element3, element4):
    result = ([element1, element2], [element3, element4])
    #result = (element1, element3)
    return result

# element now has format
# (input_re input_imag), (output_re output_imag)

dataset = dataset.map(transform_func)

show_dataset_structure()

In [31]:
if IS_TRAINING:
    FC_layer = tf.keras.layers.Dense(units=neuron_number, activation=activation_function) # dense layer == fully connected layer    

    #input_layer = tf.keras.layers.InputLayer()

    #reshape_layer = tf.keras.layers.Reshape()

    input_layer = tf.keras.layers.Dense(units=2, input_dim=2)
    output_layer = tf.keras.layers.Dense(units=2, activation=activation_function)

In [22]:
# create the model
if IS_TRAINING:
    model = tf.keras.Sequential([input_layer, FC_layer, output_layer])


In [39]:
# loss function
# as described in article: MSE mean squared error
# batches have the format:
# [batch_real, batch_imag]
def my_loss_fcn(ground_truth_batch, predicted_batch):
    print("ground_truth_batch:")
    print(ground_truth_batch)
    print("predicted_batch:")
    print(predicted_batch)
    result = tf.square(predicted_batch[:,0] - ground_truth_batch[:,0])
    result = result + tf.square(predicted_batch[:,1] - ground_truth_batch[:,1])
    N = len(ground_truth_batch)
    print("N:"+str(N))
    print(result)
    result = tf.reduce_sum(result,1) / N
    return result  

In [46]:
# compile the keras model
# loss=tf.keras.losses.MeanSquaredError()
if IS_TRAINING:
    model.compile(loss='mse', 
    optimizer=tf.keras.optimizers.Adam(learning_rate), 
    metrics=['accuracy',
    tf.keras.metrics.RootMeanSquaredError()])    

    print("Successfully compiled new model.")

Successfully compiled new model.


In [47]:
# load model from file if not training
if not IS_TRAINING:
    model = tf.keras.models.load_model("./training_" + str(trainingset_number) + "/saved_trained_model")
    print("Successfully loaded existing model.")

In [48]:
# Display the model's architecture
model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_10 (Dense)            (None, 2)                 6         
                                                                 
 dense_9 (Dense)             (None, 8)                 24        
                                                                 
 dense_11 (Dense)            (None, 2)                 18        
                                                                 
Total params: 48
Trainable params: 48
Non-trainable params: 0
_________________________________________________________________


In [49]:
# Create a callback that saves the model's weights
cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                 save_weights_only=True,
                                                 verbose=1)

# This may generate warnings related to saving the state of the optimizer.
# These warnings (and similar warnings throughout this notebook)
# are in place to discourage outdated usage, and can be ignored.

In [54]:

# fit the keras model on the dataset
#
# train the model by slicing the data into "batches" of size batch_size,
# and repeatedly iterating over the entire dataset 
# for a given number of epochs
if IS_TRAINING:
    history = model.fit(
        dataset_array_numpy[:,0:2], 
        dataset_array_numpy[:,2:4],
        callbacks=[cp_callback],
        epochs=iterations)

Epoch 1/20000
206460/206471 [============================>.] - ETA: 0s - loss: 0.0010 - accuracy: 0.6519 - root_mean_squared_error: 0.0321
Epoch 1: saving model to ./training_5\cp_test.ckpt
206471/206471 [==============================] - 575s 3ms/step - loss: 0.0010 - accuracy: 0.6519 - root_mean_squared_error: 0.0321
Epoch 2/20000
206461/206471 [============================>.] - ETA: 0s - loss: 0.0010 - accuracy: 0.6519 - root_mean_squared_error: 0.0321
Epoch 2: saving model to ./training_5\cp_test.ckpt
206471/206471 [==============================] - 577s 3ms/step - loss: 0.0010 - accuracy: 0.6519 - root_mean_squared_error: 0.0321
Epoch 3/20000
206460/206471 [============================>.] - ETA: 0s - loss: 0.0010 - accuracy: 0.6519 - root_mean_squared_error: 0.0321
Epoch 3: saving model to ./training_5\cp_test.ckpt
206471/206471 [==============================] - 634s 3ms/step - loss: 0.0010 - accuracy: 0.6519 - root_mean_squared_error: 0.0321
Epoch 4/20000
142864/206471 [========

KeyboardInterrupt: 

In [ ]:
# test the dataset against the loaded model
if not IS_TRAINING:
    model.evaluate()

In [55]:
# save the model in the correct directory
if IS_TRAINING:
    model.save("./training_" + str(trainingset_number) + "/saved_trained_model")

INFO:tensorflow:Assets written to: ./training_5/saved_trained_model\assets


In [ ]:
# The returned history object holds a record of the loss values 
# and metric values during training
print(history.history)

In [ ]:
# training progress
# diagram number 2 paper page 4

# cost function mean square error